# Final Pipeline: To generate the labels using trained classifiers

# load the models

In [ ]:
file_models_no_mask = [
    'data_2/macro_classifiers/DCBL_macro_TTV_50_SPLIT1new.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_50_SPLIT2new.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_50_SPLIT3new.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_50_SPLIT4new.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_50_SPLIT5new.hdf5'
]

In [ ]:
file_models_mask = [
    'data_2/macro_classifiers/DCBL_macro_TTV_MJW_1.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_MJW_2.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_MJW_3.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_MJW_4.hdf5',
    'data_2/macro_classifiers/DCBL_macro_TTV_MJW_5.hdf5'
]

In [ ]:
# load the models

from keras.models import load_model


models_no_mask = []
model_mask = []

for file_m in file_models_no_mask:
    print(file_m)
    model = load_model(file_m)
    models_no_mask.append(model)



for file_m in file_models_mask:
    print(file_m)
    model = load_model(file_m)
    model_mask.append(model)



# Load the final test dataset with no masks

In [ ]:
def parse_IMU_files_no_mask(parent_dir, sub_dirs, startTime, endTime, file_name, window_length):
    data = []
    
    data_count = 0
    
    for sub_dir in sub_dirs:
        channel=[]
        
        for fn in glob.glob(os.path.join(parent_dir,sub_dir, file_name)):
            file = open(fn, newline='')
            reader = csv.reader(file)
            first = True
            count = 0
            for row in reader:
                
                if first:
                    first = False
                    continue
                
                timestamp=float(row[3]) #4th column is timestamp
                if timestamp >=startTime and timestamp <=endTime and count<window_length:
                    
                    channel.append([float(row[0]),float(row[1]),float(row[2])])
                    count = count + 1 
                    data_count = data_count+1
                    
        data.append(channel)         
    return data, data_count


def parse_IMU_files_mask(parent_dir, sub_dirs, startTime, endTime, file_name, window_length):
    
    data = []
    data_mask = []
    
    data_count = 0
    
    for sub_dir in sub_dirs:
        
        channel=[[0.0, 0.0, 0.0]]*window_length
        
        mask = [0.0]*window_length
        
        div_bucket = int(10000/window_length)
        
        for fn in glob.glob(os.path.join(parent_dir,sub_dir, file_name)):
            file = open(fn, newline='')
            reader = csv.reader(file)
            first = True
            count = 0
            for row in reader:
                
                if first:
                    first = False
                    continue
                
                timestamp=float(row[3]) #4th column is timestamp
                if timestamp >=startTime and timestamp <=endTime and count<window_length:
                    
                    curr_bucket = timestamp - startTime #can go from 0 to 10,000
                    
                    curr_bucket = int(curr_bucket/div_bucket) #can go from 0 to 500
                     
                    if curr_bucket<window_length:
                        channel[curr_bucket] = [float(row[0]),float(row[1]),float(row[2])]
                        mask[curr_bucket] = 1.0

                        count = count + 1 
                        data_count = data_count+1
                    
                                       
        data.append(channel) 
        data_mask.append(mask)
    return data, data_mask, data_count

In [ ]:
import glob
import os
import numpy as np
import csv
#import cv2
import io
from sklearn.preprocessing import StandardScaler
import shutil

In [ ]:
data_min_count = 100 # per sample, valid points

In [ ]:
from scipy import stats

def get_macro_label(data_dir, sub_dirs):

    files = os.listdir(data_dir+'/left_hip')
    number_of_samples = 500

    labels_macro =dict()

    labels_macro['sandwich'] = 0
    labels_macro['fruitsalad'] = 1
    labels_macro['cereal'] = 2


    #read the labels, to verify on a sample test
    labels_loc = 'data/LabelTable.csv'
    file_label = open(labels_loc, newline='')
    label_reader = csv.reader(file_label)
    file_label_mapping = dict()

    first = True
    for row in label_reader:

        if first:
            first = False
            continue

        file_label_mapping[row[0]+'.csv'] = labels_macro[row[1]]


    total_count = 0
    correct_count = 0

    for f in files:

        file_data_mask = []
        file_data_no_mask = []
        
        file_label = []

        st_index = 0
        end_index = 30000
        step = 1000 #overlapping window, step: 1000. 
        window_index = 10000 #6 second window

        #print('reading file:',f)
        f_name = f

        if f_name == '.DS_Store':
            continue

        total_count = total_count+1

        curr_label_file = file_label_mapping[f_name]

        while st_index+step < end_index:

            data, data_mask, data_count = parse_IMU_files_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            data2, data_count = parse_IMU_files_no_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            
            
            
            st_index = st_index+step

            if data_count<data_min_count:
                continue

            data_sample_mask  = np.zeros((12, number_of_samples))
            
            data_sample_no_mask  = np.zeros((9, number_of_samples))


            data_label   = curr_label_file
            
            #print(len(data), len(data[0]), len(data_mask[0]), data_count)
            
            for i in range(len(data)):
                for j in range(len(data[i])):
                    data_sample_mask[i*4,j]=data[i][j][0]
                    data_sample_mask[i*4+1,j]=data[i][j][1]
                    data_sample_mask[i*4+2,j]=data[i][j][2]
                    
                    #mask information
                    data_sample_mask[i*4+3,j]=data_mask[i][j]

            file_data_mask.append(data_sample_mask)
            
            
            for i in range(len(data)):
                for j in range(len(data[i])):
                    data_sample_no_mask[i*3,j]=data[i][j][0]
                    data_sample_no_mask[i*3+1,j]=data[i][j][1]
                    data_sample_no_mask[i*3+2,j]=data[i][j][2]
                    

            file_data_no_mask.append(data_sample_no_mask)
            
            

        file_data_mask = np.array(file_data_mask)
        file_data_no_mask = np.array(file_data_no_mask)
        
        file_label = curr_label_file

        file_data_mask = file_data_mask.reshape((-1, 12,500, 1))
        
        file_data_no_mask = file_data_no_mask.reshape((-1, 9,500, 1))
        
        
        y_pred_final = []
        for model in models_no_mask:
            y_pred = np.argmax(model.predict(file_data_no_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
            #print(y_pred.shape)
            #print(y_pred)
        
        
        for model in model_mask:
            y_pred = np.argmax(model.predict(file_data_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
        
        y_pred_final = np.array(y_pred_final)
        
        
        #print(y_pred_final)
        #counts = np.bincount(y_pred_final)
        #prediction = np.argmax(counts) #max occuring value in windows
        prediction,count_ = stats.mode(y_pred_final)
        
        
        print('Final Prediction is:', prediction, 'file_label:', file_label)
        
        
        #correct prediction
        if int(prediction)==int(file_label):
            correct_count = correct_count+1
        
        #break
    
    return total_count, correct_count        
            

In [ ]:
data_dir_train='data_2/train'
data_dir_val= 'data_2/val'
data_dir_test= 'data_2/test'

In [ ]:
sub_dirs = ['left_hip','right_arm','right_wrist' ] #three sensors, 9 channels total

In [ ]:
# Test data accuracy:
total_count, correct_count = get_macro_label(data_dir_test, sub_dirs)
print('total_count of test files:', total_count)
print('correctly predicted test files:', correct_count)
print('Percentage accuracy:', (correct_count/total_count)*100.0)

# Final label generation: Macro

In [ ]:
from scipy import stats

def get_macro_label(data_dir, sub_dirs):

    files = os.listdir(data_dir+'/left_hip')
    number_of_samples = 500

    labels_macro =dict()

    labels_macro['sandwich'] = 0
    labels_macro['fruitsalad'] = 1
    labels_macro['cereal'] = 2
    
    labels_print=['sandwich', 'fruitsalad', 'cereal']

    total_count = 0
    correct_count = 0

    for f in files:

        file_data_mask = []
        file_data_no_mask = []
        
        file_label = []

        st_index = 0
        end_index = 30000
        step = 1000 #overlapping window, step: 1000. 
        window_index = 10000 #6 second window

        #print('reading file:',f)
        f_name = f

        if f_name == '.DS_Store':
            continue

        total_count = total_count+1

        
        while st_index+step < end_index:

            data, data_mask, data_count = parse_IMU_files_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            data2, data_count = parse_IMU_files_no_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            
            
            st_index = st_index+step

            if data_count<data_min_count:
                continue

            data_sample_mask  = np.zeros((12, number_of_samples))
            
            data_sample_no_mask  = np.zeros((9, number_of_samples))


            
            
            #print(len(data), len(data[0]), len(data_mask[0]), data_count)
            
            for i in range(len(data)):
                for j in range(len(data[i])):
                    data_sample_mask[i*4,j]=data[i][j][0]
                    data_sample_mask[i*4+1,j]=data[i][j][1]
                    data_sample_mask[i*4+2,j]=data[i][j][2]
                    
                    #mask information
                    data_sample_mask[i*4+3,j]=data_mask[i][j]

            file_data_mask.append(data_sample_mask)
            
            
            for i in range(len(data)):
                for j in range(len(data[i])):
                    data_sample_no_mask[i*3,j]=data[i][j][0]
                    data_sample_no_mask[i*3+1,j]=data[i][j][1]
                    data_sample_no_mask[i*3+2,j]=data[i][j][2]
                    

            file_data_no_mask.append(data_sample_no_mask)
            
            

        file_data_mask = np.array(file_data_mask)
        file_data_no_mask = np.array(file_data_no_mask)
        
        file_data_mask = file_data_mask.reshape((-1, 12,500, 1))
        
        file_data_no_mask = file_data_no_mask.reshape((-1, 9,500, 1))
        
        
        y_pred_final = []
        for model in models_no_mask:
            y_pred = np.argmax(model.predict(file_data_no_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
            #print(y_pred.shape)
            #print(y_pred)
        
        
        for model in model_mask:
            y_pred = np.argmax(model.predict(file_data_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
        
        y_pred_final = np.array(y_pred_final)
        
        
        #print(y_pred_final)
        #counts = np.bincount(y_pred_final)
        #prediction = np.argmax(counts) #max occuring value in windows
        prediction,count_ = stats.mode(y_pred_final)
        
        f_name_output = f_name.split('.')
        
        #last value is kind of %confidence from all the classifiers: 33% is by default
        
        print(f_name_output[0],',', labels_print[int(prediction)],',', count_/file_data_no_mask.shape[0]*10)
    

In [ ]:
data_dir_test= 'Competition_test/competition_test'

In [ ]:
sub_dirs = ['left_hip','right_arm','right_wrist' ] #three sensors, 9 channels total

In [ ]:
# Test data accuracy:
get_macro_label(data_dir_test, sub_dirs)

# Testing Macro and Micro

In [ ]:
modelcut = load_model('data_micro/DCBL_micro_Cut_jitter_mask.hdf5')
modelwash = load_model('data_micro/DCBL_micro_Wash_jitter_mask.hdf5')
modeltake = load_model('data_micro/DCBL_micro_Take_jitter_mask.hdf5')
modelput = load_model('data_micro/DCBL_micro_Put_jitter_mask.hdf5')
modelpeel = load_model('data_micro/DCBL_micro_Peel_jitter_mask.hdf5')
modeladd = load_model('data_micro/DCBL_micro_ADD_jitter_mask.hdf5')
modelmix = load_model('data_micro/DCBL_micro_Mix_jitter_mask.hdf5')
modelpour = load_model('data_micro/DCBL_micro_Pour_jitter_mask.hdf5')
modelopen = load_model('data_micro/DCBL_micro_Open_jitter_mask.hdf5')
modelother = load_model('data_micro/DCBL_micro_other_jitter_mask.hdf5')

In [ ]:
modelcut2 = load_model('data_micro/DCBL_micro_Cut_jitter.hdf5')
modelwash2 = load_model('data_micro/DCBL_micro_Wash_jitter.hdf5')
modeltake2 = load_model('data_micro/DCBL_micro_Take_jitter.hdf5')
modelput2 = load_model('data_micro/DCBL_micro_Put_jitter.hdf5')
modelpeel2 = load_model('data_micro/DCBL_micro_Peel_jitter.hdf5')
modeladd2 = load_model('data_micro/DCBL_micro_ADD_jitter.hdf5')
modelmix2 = load_model('data_micro/DCBL_micro_Mix_jitter.hdf5')
modelpour2 = load_model('data_micro/DCBL_micro_Pour_jitter.hdf5')
modelopen2 = load_model('data_micro/DCBL_micro_Open_jitter.hdf5')
modelother2 = load_model('data_micro/DCBL_micro_other_jitter.hdf5')

In [ ]:
from scipy import stats

def get_macro_label(data_dir, sub_dirs):

    files = os.listdir(data_dir+'/left_hip')
    number_of_samples = 500

    labels_macro =dict()

    labels_macro['sandwich'] = 0
    labels_macro['fruitsalad'] = 1
    labels_macro['cereal'] = 2


    #read the labels, to verify on a sample test
    labels_loc = 'data/LabelTable.csv'
    file_label = open(labels_loc, newline='')
    label_reader = csv.reader(file_label)
    file_label_mapping = dict()

    first = True
    for row in label_reader:

        if first:
            first = False
            continue

        file_label_mapping[row[0]+'.csv'] = labels_macro[row[1]]
        
        
    
    labels_micro =dict()
    labels_micro['Cut'] = 0
    labels_micro['Wash'] = 1
    labels_micro['Take'] = 2
    labels_micro['Put'] = 3
    labels_micro['Peel'] = 4
    labels_micro['Add'] = 5
    labels_micro['Mix'] = 6
    labels_micro['Pour'] = 7
    labels_micro['Open'] = 8
    labels_micro['other'] = 9 
    
    
    #read the labels
    labels_loc = 'data/LabelTable.csv'
    file_label = open(labels_loc, newline='')
    label_reader = csv.reader(file_label)
    file_label_mapping_micro = dict()
    
    for row in label_reader:
        label = row[2].split(',')
        
        label_file_micro = [0]*10
        for lab in label:
            if lab!='' and lab !='micro':
                label_file_micro[labels_micro[lab]] = 1
                
        
        file_label_mapping_micro[row[0]+'.csv'] = label_file_micro


    
    total_count = 0
    correct_count = 0
    
    correct_count_micro = 0
    
    for f in files:

        file_data_mask = []
        file_data_no_mask = []
        
        file_label = []

        st_index = 0
        end_index = 30000
        step = 1000 #overlapping window, step: 1000. 
        window_index = 10000 #6 second window

        #print('reading file:',f)
        f_name = f

        if f_name == '.DS_Store':
            continue

        total_count = total_count+1

        curr_label_file = file_label_mapping[f_name]
        
        curr_label_file_micro = file_label_mapping_micro[f_name]
        

        while st_index+step < end_index:

            data, data_mask, data_count = parse_IMU_files_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            data2, data_count = parse_IMU_files_no_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            
            
            
            st_index = st_index+step

            if data_count<data_min_count:
                continue

            data_sample_mask  = np.zeros((12, number_of_samples))
            
            data_sample_no_mask  = np.zeros((9, number_of_samples))


            data_label   = curr_label_file
            
            #print(len(data), len(data[0]), len(data_mask[0]), data_count)
            
            for i in range(len(data)):
                for j in range(len(data[i])):
                    data_sample_mask[i*4,j]=data[i][j][0]
                    data_sample_mask[i*4+1,j]=data[i][j][1]
                    data_sample_mask[i*4+2,j]=data[i][j][2]
                    
                    #mask information
                    data_sample_mask[i*4+3,j]=data_mask[i][j]

            file_data_mask.append(data_sample_mask)
            
            
            for i in range(len(data2)):
                for j in range(len(data2[i])):
                    data_sample_no_mask[i*3,j]=data2[i][j][0]
                    data_sample_no_mask[i*3+1,j]=data2[i][j][1]
                    data_sample_no_mask[i*3+2,j]=data2[i][j][2]
                    

            file_data_no_mask.append(data_sample_no_mask)
            
            

        file_data_mask = np.array(file_data_mask)
        file_data_no_mask = np.array(file_data_no_mask)
        
        file_label = curr_label_file

        file_data_mask = file_data_mask.reshape((-1, 12,500, 1))
        
        file_data_no_mask = file_data_no_mask.reshape((-1, 9,500, 1))
        
        
        y_pred_final = []
        for model in models_no_mask:
            y_pred = np.argmax(model.predict(file_data_no_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
            #print(y_pred.shape)
            #print(y_pred)
        
        
        for model in model_mask:
            y_pred = np.argmax(model.predict(file_data_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
        
        y_pred_final = np.array(y_pred_final)
        
        
        #print(y_pred_final)
        #counts = np.bincount(y_pred_final)
        #prediction = np.argmax(counts) #max occuring value in windows
        prediction_macro,count_ = stats.mode(y_pred_final)
         
        
        ##Micro prediction
        
        y_pred_cut = modelcut.predict(file_data_mask)
        y_pred_wash = modelwash.predict(file_data_mask)
        y_pred_take = modeltake.predict(file_data_mask)
        y_pred_put = modelput.predict(file_data_mask)
        y_pred_peel = modelpeel.predict(file_data_mask)
        y_pred_add = modeladd.predict(file_data_mask)
        y_pred_mix = modelmix.predict(file_data_mask)
        
        y_pred_pour = modelpour.predict(file_data_mask)
        y_pred_open = modelopen.predict(file_data_mask)
        y_pred_other = modelother.predict(file_data_mask)
                                      
        y_pred_cut2 = modelcut2.predict(file_data_no_mask)
        y_pred_wash2 = modelwash2.predict(file_data_no_mask)
        y_pred_take2 = modeltake2.predict(file_data_no_mask)
        y_pred_put2 = modelput2.predict(file_data_no_mask)
        y_pred_peel2 = modelpeel2.predict(file_data_no_mask)
        y_pred_add2 = modeladd2.predict(file_data_no_mask)
        y_pred_mix2 = modelmix2.predict(file_data_no_mask)
        y_pred_pour2 = modelpour2.predict(file_data_no_mask)
        y_pred_open2 = modelopen2.predict(file_data_no_mask)
        y_pred_other2 = modelother2.predict(file_data_no_mask)
          
            
        
        y_pred = np.zeros((file_data_mask.shape[0],10))
        
        for j in range(0,y_pred.shape[0]):
            if(y_pred_cut[j,0]>=y_pred_cut[j,1]):
                y_pred[j,0] = 1
            if(y_pred_wash[j,0]>=y_pred_wash[j,1]):
                y_pred[j,1] = 1
            if(y_pred_take[j,0]>=y_pred_take[j,1]):
                y_pred[j,2] = 1
            if(y_pred_put[j,0]>=y_pred_put[j,1]):
                y_pred[j,3] = 1
            if(y_pred_peel[j,0]>=y_pred_peel[j,1]):
                y_pred[j,4] = 1
            if(y_pred_add[j,0]>=y_pred_add[j,1]):
                y_pred[j,5] = 1
            if(y_pred_mix[j,0]>=y_pred_mix[j,1]):
                y_pred[j,6] = 1
            if(y_pred_pour[j,0]>=y_pred_pour[j,1]):
                y_pred[j,7] = 1
            if(y_pred_open[j,0]>=y_pred_open[j,1]):
                y_pred[j,8] = 1
            if(y_pred_other[j,0]>=y_pred_other[j,1]):
                y_pred[j,9] = 1
        
                                        
        y_pred2 = np.zeros((file_data_mask.shape[0],10))
        
        for j in range(0,y_pred2.shape[0]):
            if(y_pred_cut2[j,0]>=y_pred_cut2[j,1]):
                y_pred2[j,0] = 1
            if(y_pred_wash2[j,0]>=y_pred_wash2[j,1]):
                y_pred2[j,1] = 1
            if(y_pred_take2[j,0]>=y_pred_take2[j,1]):
                y_pred2[j,2] = 1
            if(y_pred_put2[j,0]>=y_pred_put2[j,1]):
                y_pred2[j,3] = 1
            if(y_pred_peel2[j,0]>=y_pred_peel2[j,1]):
                y_pred2[j,4] = 1
            if(y_pred_add2[j,0]>=y_pred_add2[j,1]):
                y_pred2[j,5] = 1
            if(y_pred_mix2[j,0]>=y_pred_mix2[j,1]):
                y_pred2[j,6] = 1
            if(y_pred_pour2[j,0]>=y_pred_pour2[j,1]):
                y_pred2[j,7] = 1
            if(y_pred_open2[j,0]>=y_pred_open2[j,1]):
                y_pred2[j,8] = 1
            if(y_pred_other2[j,0]>=y_pred_other2[j,1]):
                y_pred2[j,9] = 1
        
        #print(y_pred.shape)
        
        #print(y_pred)
        #y_pred = np.where(y_pred > 0, 1, 0)
        
        loc_arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        
        
        #uncomment for fusion
        for i in range(y_pred.shape[0]):
            loc_arr = loc_arr + y_pred[i]
                                        
        for i in range(y_pred2.shape[0]):
            loc_arr = loc_arr + y_pred2[i]
            
            
#         for i in range(y_pred2.shape[0]):
#             loc_arr = loc_arr + y_pred2[i]
        
        
        loc_arr = loc_arr/(y_pred.shape[0] + y_pred2.shape[0])
        
        #loc_arr = loc_arr/(y_pred2.shape[0])
        
        max_val = loc_arr.max()
        
        loc_arr = np.where(loc_arr > max_val/3.0, 1, 0)
        
        
        #if else logic
        
        if int(prediction_macro)==0:
            #cut=0,wash=1,take=2,put=3,other=9
            loc_arr[4]=0
            loc_arr[5]=0
            loc_arr[6]=0
            loc_arr[7]=0
            loc_arr[8]=0
        
        
        if int(prediction_macro)==1:
            #cut=0, take=2, put=3, peel=4, add=5, mix=6, other=9
            loc_arr[1]=0
            loc_arr[7]=0
            loc_arr[8]=0
        
        
        if int(prediction_macro)==2:
            #cut=0, take=2, put=3, peel=4, pour=7, open=8,  other=9
            loc_arr[1]=0
            loc_arr[5]=0
            loc_arr[6]=0 
        
        
        file_label_micro = np.array(curr_label_file_micro)
        
        
        #correct prediction
        if np.array_equal(file_label_micro, loc_arr):
            correct_count_micro = correct_count_micro+1
        
        
        
        print('Final Prediction is:', prediction_macro, 'file_label:', file_label, 'label micro',file_label_micro, loc_arr)
        
        
        #correct prediction
        if int(prediction_macro)==int(file_label):
            correct_count = correct_count+1
        
        #break
    
    return total_count, correct_count, correct_count_micro     
            

In [ ]:
# Test data accuracy:
#data_dir_test= 'data_2/test'

data_dir_test= 'test'

import time

t1 = time.time()
total_count, correct_count, correct_count_micro = get_macro_label(data_dir_test, sub_dirs)

t2 = time.time()

print(t2-t1)

In [ ]:
# print('Percentage accuracy:', (correct_count/total_count)*100.0, (correct_count_micro/total_count)*100.0)

# Final label generation of the competition test files

In [ ]:
from scipy import stats

def get_macro_label(data_dir, sub_dirs):

    files = os.listdir(data_dir+'/left_hip')
    number_of_samples = 500

    labels_print=['sandwich', 'fruitsalad', 'cereal']
    
    labels_micro_print= ['Cut','Wash','Take','Put','Peel','Add','Mix', 'Pour','Open', 'other']
    
    
    all_output_to_save = []
    
    
    labels_macro =dict()

    labels_macro['sandwich'] = 0
    labels_macro['fruitsalad'] = 1
    labels_macro['cereal'] = 2


    #read the labels, to verify on a sample test
    labels_loc = 'data/LabelTable.csv'
    file_label = open(labels_loc, newline='')
    label_reader = csv.reader(file_label)
    file_label_mapping = dict()

    first = True
    for row in label_reader:

        if first:
            first = False
            continue

        file_label_mapping[row[0]+'.csv'] = labels_macro[row[1]]
        
        
    
    labels_micro =dict()
    labels_micro['Cut'] = 0
    labels_micro['Wash'] = 1
    labels_micro['Take'] = 2
    labels_micro['Put'] = 3
    labels_micro['Peel'] = 4
    labels_micro['Add'] = 5
    labels_micro['Mix'] = 6
    labels_micro['Pour'] = 7
    labels_micro['Open'] = 8
    labels_micro['other'] = 9 
    
    
    #read the labels
    labels_loc = 'data/LabelTable.csv'
    file_label = open(labels_loc, newline='')
    label_reader = csv.reader(file_label)
    file_label_mapping_micro = dict()
    
    for row in label_reader:
        label = row[2].split(',')
        
        label_file_micro = [0]*10
        for lab in label:
            if lab!='' and lab !='micro':
                label_file_micro[labels_micro[lab]] = 1
                
        
        file_label_mapping_micro[row[0]+'.csv'] = label_file_micro


    
    total_count = 0
    correct_count = 0
    
    correct_count_micro = 0
    
    
    
    
    for f in files:

        file_data_mask = []
        file_data_no_mask = []
        
        file_label = []

        st_index = 0
        end_index = 30000
        step = 1000 #overlapping window, step: 1000. 
        window_index = 10000 #6 second window

        #print('reading file:',f)
        f_name = f

        if f_name == '.DS_Store':
            continue
            
        

        while st_index+step < end_index:

            data, data_mask, data_count = parse_IMU_files_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            data2, data_count = parse_IMU_files_no_mask(data_dir, sub_dirs, st_index, st_index+window_index,  f, number_of_samples)
            
            
            
            st_index = st_index+step

            if data_count<data_min_count:
                continue

            data_sample_mask  = np.zeros((12, number_of_samples))
            
            data_sample_no_mask  = np.zeros((9, number_of_samples))


            for i in range(len(data)):
                for j in range(len(data[i])):
                    data_sample_mask[i*4,j]=data[i][j][0]
                    data_sample_mask[i*4+1,j]=data[i][j][1]
                    data_sample_mask[i*4+2,j]=data[i][j][2]
                    
                    #mask information
                    data_sample_mask[i*4+3,j]=data_mask[i][j]

            file_data_mask.append(data_sample_mask)
            
            
            for i in range(len(data2)):
                for j in range(len(data2[i])):
                    data_sample_no_mask[i*3,j]=data2[i][j][0]
                    data_sample_no_mask[i*3+1,j]=data2[i][j][1]
                    data_sample_no_mask[i*3+2,j]=data2[i][j][2]
                    

            file_data_no_mask.append(data_sample_no_mask)
            
            

        file_data_mask = np.array(file_data_mask)
        file_data_no_mask = np.array(file_data_no_mask)
        
        #file_label = curr_label_file

        file_data_mask = file_data_mask.reshape((-1, 12,500, 1))
        
        file_data_no_mask = file_data_no_mask.reshape((-1, 9,500, 1))
        
        
        y_pred_final = []
        for model in models_no_mask:
            y_pred = np.argmax(model.predict(file_data_no_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
            #print(y_pred.shape)
            #print(y_pred)
        
        
        for model in model_mask:
            y_pred = np.argmax(model.predict(file_data_mask), axis=1)
            
            for i in range(y_pred.shape[0]):
                y_pred_final.append(y_pred[i])
        
        y_pred_final = np.array(y_pred_final)
        
        
        #print(y_pred_final)
        #counts = np.bincount(y_pred_final)
        #prediction = np.argmax(counts) #max occuring value in windows
        prediction_macro,count_ = stats.mode(y_pred_final)
         
        
        ##Micro prediction
        
        y_pred_cut = modelcut.predict(file_data_mask)
        y_pred_wash = modelwash.predict(file_data_mask)
        y_pred_take = modeltake.predict(file_data_mask)
        y_pred_put = modelput.predict(file_data_mask)
        y_pred_peel = modelpeel.predict(file_data_mask)
        y_pred_add = modeladd.predict(file_data_mask)
        y_pred_mix = modelmix.predict(file_data_mask)
        
        y_pred_pour = modelpour.predict(file_data_mask)
        y_pred_open = modelopen.predict(file_data_mask)
        y_pred_other = modelother.predict(file_data_mask)
                                      
        y_pred_cut2 = modelcut2.predict(file_data_no_mask)
        y_pred_wash2 = modelwash2.predict(file_data_no_mask)
        y_pred_take2 = modeltake2.predict(file_data_no_mask)
        y_pred_put2 = modelput2.predict(file_data_no_mask)
        y_pred_peel2 = modelpeel2.predict(file_data_no_mask)
        y_pred_add2 = modeladd2.predict(file_data_no_mask)
        y_pred_mix2 = modelmix2.predict(file_data_no_mask)
        y_pred_pour2 = modelpour2.predict(file_data_no_mask)
        y_pred_open2 = modelopen2.predict(file_data_no_mask)
        y_pred_other2 = modelother2.predict(file_data_no_mask)
          
            
        
        y_pred = np.zeros((file_data_mask.shape[0],10))
        
        for j in range(0,y_pred.shape[0]):
            if(y_pred_cut[j,0]>=y_pred_cut[j,1]):
                y_pred[j,0] = 1
            if(y_pred_wash[j,0]>=y_pred_wash[j,1]):
                y_pred[j,1] = 1
            if(y_pred_take[j,0]>=y_pred_take[j,1]):
                y_pred[j,2] = 1
            if(y_pred_put[j,0]>=y_pred_put[j,1]):
                y_pred[j,3] = 1
            if(y_pred_peel[j,0]>=y_pred_peel[j,1]):
                y_pred[j,4] = 1
            if(y_pred_add[j,0]>=y_pred_add[j,1]):
                y_pred[j,5] = 1
            if(y_pred_mix[j,0]>=y_pred_mix[j,1]):
                y_pred[j,6] = 1
            if(y_pred_pour[j,0]>=y_pred_pour[j,1]):
                y_pred[j,7] = 1
            if(y_pred_open[j,0]>=y_pred_open[j,1]):
                y_pred[j,8] = 1
            if(y_pred_other[j,0]>=y_pred_other[j,1]):
                y_pred[j,9] = 1
        
                                        
        y_pred2 = np.zeros((file_data_no_mask.shape[0],10))
        
        for j in range(0,y_pred2.shape[0]):
            if(y_pred_cut2[j,0]>=y_pred_cut2[j,1]):
                y_pred2[j,0] = 1
            if(y_pred_wash2[j,0]>=y_pred_wash2[j,1]):
                y_pred2[j,1] = 1
            if(y_pred_take2[j,0]>=y_pred_take2[j,1]):
                y_pred2[j,2] = 1
            if(y_pred_put2[j,0]>=y_pred_put2[j,1]):
                y_pred2[j,3] = 1
            if(y_pred_peel2[j,0]>=y_pred_peel2[j,1]):
                y_pred2[j,4] = 1
            if(y_pred_add2[j,0]>=y_pred_add2[j,1]):
                y_pred2[j,5] = 1
            if(y_pred_mix2[j,0]>=y_pred_mix2[j,1]):
                y_pred2[j,6] = 1
            if(y_pred_pour2[j,0]>=y_pred_pour2[j,1]):
                y_pred2[j,7] = 1
            if(y_pred_open2[j,0]>=y_pred_open2[j,1]):
                y_pred2[j,8] = 1
            if(y_pred_other2[j,0]>=y_pred_other2[j,1]):
                y_pred2[j,9] = 1
        
        #print(y_pred.shape)
        
        #print(y_pred)
        #y_pred = np.where(y_pred > 0, 1, 0)
        
        loc_arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        
        
        #uncomment for fusion
        for i in range(y_pred.shape[0]):
            loc_arr = loc_arr + y_pred[i]
                                        
        for i in range(y_pred2.shape[0]):
            loc_arr = loc_arr + y_pred2[i]
            
            
#         for i in range(y_pred.shape[0]):
#             loc_arr = loc_arr + y_pred[i]
        
        
        loc_arr = loc_arr/(y_pred.shape[0] + y_pred2.shape[0])
        
        #loc_arr = loc_arr/(y_pred.shape[0])
        
        max_val = loc_arr.max()
        
        loc_arr = np.where(loc_arr > max_val/3.0, 1, 0)
        
        
        
        f_name_output = f_name.split('.')
        
        
        if int(prediction_macro)==0:
            #cut=0,wash=1,take=2,put=3,other=9
            loc_arr[4]=0
            loc_arr[5]=0
            loc_arr[6]=0
            loc_arr[7]=0
            loc_arr[8]=0
        
        
        if int(prediction_macro)==1:
            #cut=0, take=2, put=3, peel=4, add=5, mix=6, other=9
            loc_arr[1]=0
            loc_arr[7]=0
            loc_arr[8]=0
        
        
        if int(prediction_macro)==2:
            #cut=0, take=2, put=3, peel=4, pour=7, open=8,  other=9
            loc_arr[1]=0
            loc_arr[5]=0
            loc_arr[6]=0 
        
        output_micro = ''
        
        for i in range(loc_arr.shape[0]):
            if loc_arr[i]==1:
                output_micro = output_micro+','+labels_micro_print[i] 
        
        all_output_to_save.append([f_name_output[0],labels_print[int(prediction_macro)], output_micro])
        
        
        
        
        
        curr_label_file = file_label_mapping[f_name]
        
        curr_label_file_micro = file_label_mapping_micro[f_name]
        
        #correct prediction
        if int(prediction_macro)==int(curr_label_file):
            correct_count = correct_count+1
        
        if np.array_equal(curr_label_file_micro, loc_arr):
            correct_count_micro = correct_count_micro+1
        
        print(correct_count, correct_count_micro)
        
    #return all_output_to_save
    return total_count, correct_count, correct_count_micro     
            

In [ ]:
#data_dir_test= 'Competition_test/competition_test'

data_dir_test= 'data_2/test'

In [ ]:
sub_dirs = ['left_hip','right_arm','right_wrist' ] #three sensors, 9 channels total

In [ ]:
total_count, correct_count, correct_count_micro = get_macro_label(data_dir_test, sub_dirs)

In [ ]:
print('Percentage accuracy:', (correct_count/50)*100.0, (correct_count_micro/50)*100.0)